<a href="https://colab.research.google.com/github/aiaus/RAG_for_Friends/blob/main/RAG_for_Friends_TV_show.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Install and import packages

In [ ]:
pip install sentence_transformers chromadb==0.4.5 ctransformers==0.2.13 langchain

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline
from langchain.llms import HuggingFaceHub


# Import Friends' wikipedia page and apply text embedding

In [ ]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Friends")
data = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
all_splits = text_splitter.split_documents(data)



# Create RAG's embedding

In [ ]:
# apply string
pretrained_embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2', model_kwargs={'device': 'cuda'})

# apply text clearning and embeddings
vectorstore= Chroma.from_documents(documents=all_splits, embedding=pretrained_embeddings)


In [ ]:
# check an sample of embeddings
query = "Who plays Ross?"

docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

Having worked with David Schwimmer in the past,[58] the series creators wrote the character of Ross with him in mind, and he was the first actor cast.[60] Cox wanted to play the role of Monica because she liked the "strong" character, but the producers had her in mind to play Rachel because of her "cheery, upbeat energy", which was not how they envisioned Monica; after Cox's audition, though, Kauffman agreed with Cox, and she got the role.[49][61] When Matt LeBlanc auditioned for Joey, he put a


In [ ]:
# Define LLM model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,pipeline
from langchain import HuggingFacePipeline

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs={"temperature": 0.2, "max_length": 512},
)

from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(template=prompt_template,
                                               input_variables=["context", "question"])


from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="stuff",
  retriever=vectorstore.as_retriever(),
  chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)





/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
question = "Who plays Ross in Friends?"
result = qa_chain({"query": question})
# Check the result of the query
result["result"]


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Token indices sequence length is longer than the specified ma

'David Schwimmer'